# NLP for Full_Text
coba biar lebih rapi

## Import Library

In [5]:
# ===== Import Library =====
import re
import json
import pandas as pd
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import nltk

# pastikan nltk stopwords tersedia
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/spil/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Load Dataset & Path

In [7]:
# ===== Load & Clean Dataset =====
import pandas as pd

# Path file
#file_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/Cleaned_1.csv'
#output_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/Cleaned_DropNull.csv'

file_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/29k_formatted.csv'
output_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/29k_formatted.csv'

# 1. Load dataset
df = pd.read_csv(file_path)
print("✅ Dataset Loaded Successfully")
print(f"Jumlah baris awal: {len(df)}")
print(f"Kolom tersedia: {list(df.columns)}")

# 2. Cari baris invalid/null
invalid_rows = df[
    df['image_corelation'].isna() | (df['image_corelation'] == '-') |
    df['Label'].isna() | (df['Label'] == '-')
]

print("/n🔍 Pengecekan data invalid:")
print("Jumlah baris invalid unik:", len(invalid_rows))

# 3. Mask valid rows
mask = (
    df['image_corelation'].notna() & (df['image_corelation'] != '-') &
    df['Label'].notna() & (df['Label'] != '-')
)

# 4. Terapkan mask → dataset cleaned
df_cleaned = df[mask].copy()

print("/n📊 Ringkasan pembersihan:")
print("Jumlah baris sebelum drop:", len(df))
print("Jumlah baris sesudah drop:", len(df_cleaned))
print("Jumlah baris yang dihapus:", len(df) - len(df_cleaned))

# 5. Simpan dataset hasil cleaning
df_cleaned.to_csv(output_path, index=False)
print(f"/n💾 Dataset cleaned berhasil disimpan ke {output_path}")

# Preview
df_cleaned.head(5)


✅ Dataset Loaded Successfully
Jumlah baris awal: 29432
Kolom tersedia: ['conversation_id_str', 'created_at', 'favorite_count', 'full_text', 'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 'retweet_count', 'tweet_url', 'user_id_str', 'username', 'official_account', 'age_account', 'verified_account', 'followers_count', 'image_corelation', 'Label', 'MM/DD/YYYY']
/n🔍 Pengecekan data invalid:
Jumlah baris invalid unik: 13913
/n📊 Ringkasan pembersihan:
Jumlah baris sebelum drop: 29432
Jumlah baris sesudah drop: 15519
Jumlah baris yang dihapus: 13913
/n💾 Dataset cleaned berhasil disimpan ke /home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/29k_formatted.csv


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,...,tweet_url,user_id_str,username,official_account,age_account,verified_account,followers_count,image_corelation,Label,MM/DD/YYYY
1,1915274254142365909,2025-04-24 5:19:00,21,Kamis Siang Genkz Selaen KORUPSI adalah NARKOB...,1915274254142365909,https://pbs.twimg.com/media/GpRrx9tbYAUriQq.jpg,NaN,in,Indonesia,1,...,https://x.com/_cenk90E/status/1915274254142365909,1316816732,_cenk90E,0,12 Tahun 6 Bulan,0,8738.0,1,1,2013-03-01
2,1910193214877003980,2025-04-10 4:48:46,17,Cara Main Saham US &amp; Saham di $IHSG Tahun ...,1910193214877003980,https://pbs.twimg.com/media/GoJemIYa4AA9bz7.jpg,NaN,in,Salatiga - Solo - DKI Jakarta,1,...,https://x.com/_fireshare/status/19101932148770...,145039939,_fireshare,0,15 Tahun 4 Bulan,0,1952.0,1,0,2010-05-01
3,1910251576784285827,2025-04-10 11:03:08,1,Jadi besok ada 2 tema deg-degan buat anak saha...,1910287427211567174,https://pbs.twimg.com/media/GoK0SdXbsAAbYYd.jpg,_fireshare,in,Salatiga - Solo - DKI Jakarta,0,...,https://x.com/_fireshare/status/19102874272115...,145039939,_fireshare,0,15 Tahun 4 Bulan,0,1951.0,1,1,2010-05-01
4,1909542732504482280,2025-04-08 9:43:59,108,Bukan perihal angkanya. Membelokkan fakta sebe...,1909542732504482280,https://pbs.twimg.com/ext_tw_video_thumb/19095...,NaN,in,NaN,0,...,https://x.com/_riverheaven/status/190954273250...,975389335775199232,_riverheaven,0,7 Tahun 6 Bulan,0,46800.0,1,0,2018-03-01
5,1909511684500361701,2025-04-08 7:45:00,0,IHSG besok sekali lagi ya https://t.co/9n7DeswnmN,1909512787711476154,https://pbs.twimg.com/media/Gn_zwPxbwAA43An.jpg,0xdermawan,in,Indonesia,0,...,https://x.com/0xdermawan/status/19095127877114...,1521895894391885824,0xdermawan,0,3 Tahun 4 Bulan,1,606.0,1,0,2022-05-01


## Cleaning

In [23]:
# ===== Cleaning =====
def cleaning_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)               # hapus URL
    text = re.sub(r'@\w+|#\w+', '', text)                    # hapus mention & hashtag
    text = re.sub(r'(\d+)\s*-\s*(\d+)', r'\1_\2', text)      # ganti range angka jadi 3_5
    text = re.sub(r'\b[a-z]\b', '', text, flags=re.IGNORECASE)  # hapus kata 1 huruf
    text = re.sub(r'-\b', '', text)                          # hilangkan strip di akhir kata
    text = re.sub(r'[^a-zA-Z0-9\s_]', ' ', text)             # hapus simbol/emoji
    text = re.sub(r'\s+', ' ', text).strip()                 # rapikan spasi
    text = re.sub(r'\b\d+\b', '', text)                      # Hapus angka berdiri sendiri (tahun, angka random, 000, dsb.)
    text = re.sub(r'\bamp\b', '', text, flags=re.IGNORECASE)   # hapus 'amp'
    return text


tqdm.pandas(desc="Cleaning")
df_cleaned['cleaned'] = df_cleaned['full_text'].progress_apply(cleaning_text)

df_cleaned[['full_text','cleaned']].head(5)


Cleaning: 100%|██████████| 15519/15519 [00:01<00:00, 8195.13it/s] 


,full_text,cleaned
1,Kamis Siang Genkz Selaen KORUPSI adalah NARKOB...,Kamis Siang Genkz Selaen KORUPSI adalah NARKOB...
2,Cara Main Saham US &amp; Saham di $IHSG Tahun ...,Cara Main Saham US Saham di IHSG Tahun
3,Jadi besok ada 2 tema deg-degan buat anak saha...,Jadi besok ada tema degdegan buat anak saham ...
4,Bukan perihal angkanya. Membelokkan fakta sebe...,Bukan perihal angkanya Membelokkan fakta seben...
5,IHSG besok sekali lagi ya https://t.co/9n7DeswnmN,IHSG besok sekali lagi ya


## Case Folding

In [24]:
# ===== Case Folding =====
def case_folding(text):
    return text.lower()

tqdm.pandas(desc="Case Folding")
df_cleaned['casefold'] = df_cleaned['cleaned'].progress_apply(case_folding)

df_cleaned[['cleaned','casefold']].head(5)


Case Folding: 100%|██████████| 15519/15519 [00:00<00:00, 513707.82it/s]


,cleaned,casefold
1,Kamis Siang Genkz Selaen KORUPSI adalah NARKOB...,kamis siang genkz selaen korupsi adalah narkob...
2,Cara Main Saham US Saham di IHSG Tahun,cara main saham us saham di ihsg tahun
3,Jadi besok ada tema degdegan buat anak saham ...,jadi besok ada tema degdegan buat anak saham ...
4,Bukan perihal angkanya Membelokkan fakta seben...,bukan perihal angkanya membelokkan fakta seben...
5,IHSG besok sekali lagi ya,ihsg besok sekali lagi ya


## Tokenization

In [25]:
# ===== Tokenization =====
def tokenize_text(text):
    return text.split()

tqdm.pandas(desc="Tokenization")
df_cleaned['tokenized'] = df_cleaned['casefold'].progress_apply(tokenize_text)

df_cleaned[['casefold','tokenized']].head(5)


Tokenization:   0%|          | 0/15519 [00:00<?, ?it/s]

Tokenization: 100%|██████████| 15519/15519 [00:00<00:00, 107820.60it/s]


,casefold,tokenized
1,kamis siang genkz selaen korupsi adalah narkob...,"[kamis, siang, genkz, selaen, korupsi, adalah,..."
2,cara main saham us saham di ihsg tahun,"[cara, main, saham, us, saham, di, ihsg, tahun]"
3,jadi besok ada tema degdegan buat anak saham ...,"[jadi, besok, ada, tema, degdegan, buat, anak,..."
4,bukan perihal angkanya membelokkan fakta seben...,"[bukan, perihal, angkanya, membelokkan, fakta,..."
5,ihsg besok sekali lagi ya,"[ihsg, besok, sekali, lagi, ya]"


## Normalization

In [26]:
# ===== Normalisasi Slang =====
slang_path = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/Kamus/combined_slang_words.json"
with open(slang_path,'r') as f:
    slang_dict = json.load(f)

def normalize_tokens(tokens, slang_dict):
    return [slang_dict.get(w,w) for w in tokens]

tqdm.pandas(desc="Normalization")
df_cleaned['normalized'] = df_cleaned['tokenized'].progress_apply(lambda x: normalize_tokens(x, slang_dict))

df_cleaned[['tokenized','normalized']].head(5)


Normalization:   0%|          | 0/15519 [00:00<?, ?it/s]

Normalization: 100%|██████████| 15519/15519 [00:00<00:00, 64620.13it/s]


,tokenized,normalized
1,"[kamis, siang, genkz, selaen, korupsi, adalah,...","[kamis, siang, teman, selaen, korupsi, adalah,..."
2,"[cara, main, saham, us, saham, di, ihsg, tahun]","[cara, main, saham, us, saham, di, ihsg, tahun]"
3,"[jadi, besok, ada, tema, degdegan, buat, anak,...","[jadi, besok, ada, tema, degdegan, buat, anak,..."
4,"[bukan, perihal, angkanya, membelokkan, fakta,...","[bukan, perihal, angkanya, membelokkan, fakta,..."
5,"[ihsg, besok, sekali, lagi, ya]","[ihsg, besok, sekali, lagi, iya]"


## Stopwords Removal

In [27]:
# ===== Stopword Removal =====
stopwords_path = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/Kamus/combined_stop_words.txt"
with open(stopwords_path,'r') as f:
    custom_stop = set([line.strip() for line in f])

nltk_stop = set(stopwords.words('indonesian'))
all_stop = nltk_stop.union(custom_stop)

def remove_stopwords(tokens, stopset):
    return [w for w in tokens if w not in stopset]

tqdm.pandas(desc="Stopword Removal")
df_cleaned['stop_removed'] = df_cleaned['normalized'].progress_apply(lambda x: remove_stopwords(x, all_stop))

df_cleaned[['normalized','stop_removed']].head(5)


Stopword Removal: 100%|██████████| 15519/15519 [00:00<00:00, 70958.08it/s]


,normalized,stop_removed
1,"[kamis, siang, teman, selaen, korupsi, adalah,...","[kamis, siang, teman, selaen, korupsi, narkoti..."
2,"[cara, main, saham, us, saham, di, ihsg, tahun]","[main, saham, us, saham, ihsg]"
3,"[jadi, besok, ada, tema, degdegan, buat, anak,...","[besok, tema, degdegan, anak, saham, ihsg, exd..."
4,"[bukan, perihal, angkanya, membelokkan, fakta,...","[perihal, angkanya, membelokkan, fakta, bangsa..."
5,"[ihsg, besok, sekali, lagi, iya]","[ihsg, besok]"


#### CHECKING Sebelum STEMMING
abaikan cell ini kalo dataset udah fix 
run cell ini kalo mau kurangi data buat masuk stemming (karena proses stemming itu lama)

In [39]:
from collections import Counter

### NGECEK TOP WORDS YANG MUNCUL SETELAH STOPWORD REMOVAL
def show_top_words(df, column_name='stop_removed', top_n=20):

    # Gabungkan semua token jadi satu list besar
    all_words = [word for tokens in df[column_name] for word in tokens if isinstance(tokens, list)]
    
    # Hitung frekuensi kata
    word_counts = Counter(all_words)
    
    # Ambil top N kata
    top_words = word_counts.most_common(top_n)
    
    # Konversi ke DataFrame agar mudah dibaca/visualisasi
    top_df = pd.DataFrame(top_words, columns=['word', 'count'])
    print(top_df)
    return top_df

topn = show_top_words(df_cleaned, column_name='stop_removed', top_n=50)


           word  count
0          ihsg   2210
1     danantara   1841
2        rupiah   1064
3     indonesia    781
4         saham    656
5       ekonomi    539
6     investasi    496
7        negara    321
8         turun    289
9          bumn    272
10        pasar    252
11        calya    237
12      menguat    228
13         live    223
14       toyota    223
15       seribu    218
16        harga    217
17       shopee    210
18      melemah    209
19      prabowo    207
20         sale    199
21        flash    195
22         aset    194
23         wave    190
24        nilai    183
25     presiden    181
26         sesi    173
27        asing    171
28        merah    170
29        level    168
30       sektor    160
31     investor    158
32          jam    158
33    penutupan    154
34     nasional    154
35        mobil    154
36  pertumbuhan    152
37       indeks    151
38          sih    148
39      ditutup    147
40        malam    147
41         dana    146
42         

In [37]:
### CEK LABEL DISTRIBUSI
def drop_null_and_check_labels(df, text_col='full_text', label_col='Label'):
    """
    Drop baris null dan tampilkan proporsi label.
    """
    df = df.dropna(subset=[text_col, label_col]).reset_index(drop=True)
    label_counts = df[label_col].value_counts().sort_index()
    label_ratio = (label_counts / len(df) * 100).round(2)

    print(f"Total data setelah drop null: {len(df)}\n")
    print("Distribusi label:")
    for label, count in label_counts.items():
        print(f"{label}: {count} data ({label_ratio[label]}%)")

    return df


def sample_balanced_data(df, label_col='Label', n_total=5000, random_state=42):
    """
    Ambil sample data dengan proporsi label seimbang.
    """
    labels = df[label_col].unique()
    n_per_label = n_total // len(labels)

    samples = []
    for label in labels:
        subset = df[df[label_col] == label]
        samples.append(subset.sample(n=min(n_per_label, len(subset)), random_state=random_state))
    
    df_sampled = pd.concat(samples).sample(frac=1, random_state=random_state).reset_index(drop=True)

    label_counts = df_sampled[label_col].value_counts().sort_index()
    label_ratio = (label_counts / len(df_sampled) * 100).round(2)

    print(f"Total data sampled: {len(df_sampled)}")
    print("Distribusi label:")
    for label, count in label_counts.items():
        print(f"{label}: {count} data ({label_ratio[label]}%)")

    return df_sampled


# Eksekusi
print("\nPengecekan data setelah cleaning...")
df_sampled = drop_null_and_check_labels(df_cleaned, 'full_text', 'Label')

print("\nSampling data dengan proporsi label seimbang...")
df_cleaned = sample_balanced_data(df_cleaned, label_col='Label', n_total=5000)



Pengecekan data setelah cleaning...
Total data setelah drop null: 15519

Distribusi label:
0: 6812 data (43.89%)
1: 8707 data (56.11%)

Sampling data dengan proporsi label seimbang...
Total data sampled: 5000
Distribusi label:
0: 2500 data (50.0%)
1: 2500 data (50.0%)


## Stemming

In [38]:
# ===== Stemming =====
exceptions_path = r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/Kamus/combined_exceptions.txt"
with open(exceptions_path,'r') as f:
    exception_list = set([line.strip() for line in f if line.strip()])

stemmer = StemmerFactory().create_stemmer()

def stemming_tokens(tokens, stemmer, exceptions):
    result = []
    for w in tokens:
        if w in exceptions:
            result.append(w)
        else:
            result.append(stemmer.stem(w))
    return result

tqdm.pandas(desc="Stemming")
df_cleaned['stemmed'] = df_cleaned['stop_removed'].progress_apply(lambda x: stemming_tokens(x, stemmer, exception_list))

df_cleaned[['stop_removed','stemmed']].head(5)


Stemming:   0%|          | 0/5000 [00:00<?, ?it/s]

Stemming: 100%|██████████| 5000/5000 [25:22<00:00,  3.28it/s]  


,stop_removed,stemmed
0,"[tanggung2, poin, ihsg, terbang, melulu, kena,...","[tanggung2, poin, ihsg, terbang, melulu, kena,..."
1,"[asumsi, wave, merah, selesai, area, 7454_7347...","[asumsi, wave, merah, selesai, area, 7454 7347..."
2,"[shm, ihsg, ditutup, hijau, diakui, king, pula...","[shm, ihsg, tutup, hijau, aku, king, pulang pe..."
3,"[penutupan, sesi, maret, ihsg]","[tutup, sesi, maret, ihsg]"
4,"[ente, bilang, kedepannya, kedepan, ihsg, anjl...","[ente, bilang, depan, depan, ihsg, anjlok, sha..."


## Detokenize

In [14]:
# ===== Detokenize =====
from nltk.tokenize.treebank import TreebankWordDetokenizer

def detokenize(tokens):
    return TreebankWordDetokenizer().detokenize(tokens)

tqdm.pandas(desc="Detokenize")
df_cleaned['clean_text'] = df_cleaned['stemmed'].progress_apply(detokenize)

df_cleaned[['full_text','clean_text']].head(5)


Detokenize: 100%|██████████| 16526/16526 [00:00<00:00, 20105.76it/s]


,full_text,clean_text
0,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1 mulyono final tokoh korup dunia 2024 2 negar...
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,saham indonesia bluchip iya bagus pair nya jel...
2,Pejuang Rupiah! https://t.co/MyCuEUKTfP,juang rupiah
3,IHSG menuju 5000 https://t.co/6FlyFblaMt,ihsg 5000
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,rupiah dadak perkasa level rp 8170 dolar as ca...


## Save Final File

In [15]:
# ===== Save Final Result =====
cleaned_text_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/Cleaned_Text.csv'
save_copy = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/Cleaned_Text_Backup.csv'

susunan_kolom = [
    'full_text',
    'clean_text',
    'tweet_url',
    'image_url',
    'user_id_str',
    'conversation_id_str',
    'created_at',
    'MM/DD/YYYY',
    'age_account',
    'username',
    'official_account',
    'verified_account',
    'quote_count',
    'reply_count',
    'retweet_count',
    'favorite_count',
    'followers_count',
    'image_corelation',
    'Label'
]

#susunan_kolom = ['full_text','clean_text','image_corelation','Label']
final_df = df_cleaned[susunan_kolom]

final_df.to_csv(cleaned_text_path, index=False)
final_df.to_csv(save_copy, index=False)

print(f"Preprocessing selesai! Hasil disimpan di: {cleaned_text_path}")

final_df.head(5)


Preprocessing selesai! Hasil disimpan di: /home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/Cleaned_Text.csv


,full_text,clean_text,tweet_url,image_url,user_id_str,conversation_id_str,created_at,MM/DD/YYYY,age_account,username,official_account,verified_account,quote_count,reply_count,retweet_count,favorite_count,followers_count,image_corelation,Label
0,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1 mulyono final tokoh korup dunia 2024 2 negar...,https://x.com/undefined/status/187482598741610...,https://pbs.twimg.com/ext_tw_video_thumb/18748...,1159689173340585984,1874825987416109464,Thu Jan 02 14:32:02 +0000 2025,08/01/2019,6 Tahun 8 Bulan,__AnakKolong,1,1,3,66,138,494,91600.0,0,1
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,saham indonesia bluchip iya bagus pair nya jel...,https://x.com/undefined/status/189432670738733...,https://pbs.twimg.com/media/GkoAGfWbkAYpK7_.jpg,2801067998,1894326707387338853,Tue Feb 25 10:00:56 +0000 2025,09/01/2014,11 Tahun 8 Bulan,__Anto_77_,0,0,1,1,0,0,695.0,1,0
2,Pejuang Rupiah! https://t.co/MyCuEUKTfP,juang rupiah,https://x.com/undefined/status/189539764466470...,https://pbs.twimg.com/media/Gk3OGjPaoAAvMJF.jpg,1848923371222011904,1895397644664705226,Fri Feb 28 08:56:27 +0000 2025,10/01/2024,1 Tahun 8 Bulan,__BADjingan,0,0,0,0,0,0,133.0,0,0
3,IHSG menuju 5000 https://t.co/6FlyFblaMt,ihsg 5000,https://x.com/undefined/status/190270849377549...,https://pbs.twimg.com/media/GmfHSZiagAAycIu.jpg,442517900,1902708493775491221,2025-03-20 13:07:10,2011-12-01,13 Tahun 9 Bulan,__badkisser,0,0,0,0,0,0,353.0,1,0
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,rupiah dadak perkasa level rp 8170 dolar as ca...,https://x.com/undefined/status/188569236798405...,https://pbs.twimg.com/media/GitTNLsaAAApkEu.jpg,799023207294504960,1885692367984054742,Sat Feb 01 14:11:09 +0000 2025,11/01/2016,9 Tahun 8 Bulan,__LOVE_AG4EVER,1,1,0,2,12,23,128000.0,1,1
